In [1]:
using Arpack
using LinearAlgebra
using IterativeSolvers
using SparseArrays

In [2]:
function read_mat()
    Is = Int64[]
    Js = Int64[]
    # data downloaded from http://snap.stanford.edu/data/enwiki-2013.html
    open("enwiki-2013.txt") do f
        for line in eachline(f)
            if line[1] == '#'; continue; end # skip lines starting with #
            i, j = split(line)
            # note 0 --> 1 indexing
            push!(Is, parse(Int64, i) + 1)
            push!(Js, parse(Int64, j) + 1)
        end
    end
    n = max(maximum(Is), maximum(Js))
    A = sparse(Is, Js, 1, n, n)
    A = min.(A, 1)
    A = max.(A, A')  # symmetrize
    d = vec(sum(A, dims=1))
    # Skip unused indices
    keep = d .> 0
    return A[keep, keep]
end

read_mat (generic function with 1 method)

In [3]:
@time A = read_mat()  # takes about one minute
d = vec(sum(A, dims=1))
α = 0.85
D = Diagonal(d)
S = D - α * A
;

 75.713478 seconds (405.49 M allocations: 35.570 GiB, 4.69% gc time)


In [4]:
n = size(S, 1)
n, nnz(S)

(4203323, 188082777)

In [5]:
# Still have fast matmuls!
x = randn(n)
@time S * x;

  0.863799 seconds (72.46 k allocations: 35.949 MiB)


In [6]:
# Splitting method for Ax = b
true_x = randn(n)
b = S * true_x
x = zeros(n)
for k = 1:20
    y = α * (A * x) + b
    x = D \ y
    println(norm(x - true_x))
end

522.1935433533063
89.83676750158715
78.97087260206708
22.173206092975718
32.27016750641598
10.600438671343358
15.72746211359596
5.958262666549539
8.177252374415165
3.539116569597201
4.425704718783172
2.1571393003847206
2.4716787265190856
1.3362398659081585
1.4194707427040794
0.8391803832154308
0.8373874592957476
0.5345217769790489
0.5074853626328737
0.34574245199858494


In [7]:
# Set up LLS problem
p = 100000
S1 = S[:, 1:p];
y = randn(p)
b = S1 * y
b = b ./ norm(b)
r = rand(n)
r = (r / norm(r)) .* 1e-5
b1 = b + r
;

In [8]:
# Iterative solver for least squares
@time z = lsmr(S1, b1, atol=1e-4, btol=1e-4);

 16.353536 seconds (1.68 M allocations: 363.052 MiB, 0.75% gc time)


In [9]:
# Relative error in LLS
norm(S1 * z - b1) / norm(b1)

0.06121447988220485

In [10]:
# Iterative solver for two largest magnitude eigenvalues and eigenvectors
@time Λ, Z = eigs(S, tol=1e-7, nev=2, which=:LM)

 37.567399 seconds (6.19 M allocations: 1.150 GiB, 0.26% gc time)


([432260.722607671, 168651.72267648167], [-4.37124351770321e-19 -4.5976163781745166e-17; -2.020486045137181e-20 -3.7637556190508666e-18; … ; 1.068194728367743e-18 1.2111204191659628e-16; 3.9538262958402107e-19 4.6752269468706075e-17], 2, 2, 37, [3.231093812979701, 0.2657767987126822, -6.218425478877261, -7.964108215884627, 0.6100111069833637, -2.3509833017340167, 3.3734583691956455, -2.1353562521949745, 2.6531912842611587, -1.267843035428465  …  0.3010695427328299, 3.21114150735015, 3.764493405517252, -3.985246663514036, -8.707910891072236, 3.2260424133276597, 0.7592611288862899, 5.997884802307534, -8.507260913334559, -3.2825411074852706])

In [ ]:
# Error in eigenvectors
norm(S * Z - Z * Diagonal(Λ)), norm(Z'Z - I)

In [ ]:
norm(S * Z[:, 1] - Z[:, 1] * Λ[1])

In [ ]:
norm(S * Z[:, 2] - Z[:, 2] * Λ[2])